In [1]:
import kfp.dsl as dsl
import kfp.gcp as gcp
import kfp.compiler as compiler
from kfp import components
from kubernetes import client as k8s_client

In [4]:
def git_clone_op(repo_url, password):
    volume_op = dsl.VolumeOp(
        name='create pipeline volume',
        resource_name='pipeline-pvc',
        modes=['ReadWriteMany'],
        size="1Gi"
    )

    image = 'alpine/git:latest'

    commands = [
        f"git clone {repo_url} /home/jovyan/vq-vae_1d",
        f"cd /home/jovyan/vq-vae_1d",
        "apk --update add sshpass",
        # "apt-get -y install sshpass",
        "mkdir ~/.ssh",
        # 'echo "Host *" >> ~/.ssh/config',
        # 'echo "StrictHostKeyChecking no" >> ~/.ssh/config',
        "sshpass -p 'House101' scp -r -o StrictHostKeyChecking=no k_ohmori@192.168.11.107:~/hdd/ohmori/data .",
        "rm -rf /var/lib/apt/lists/*",
        "rm /var/cache/apk/*",
        "pwd",
        'ls',
    ]

    # Mount Git repository in /workspace
    op = dsl.ContainerOp(
        name='git clone',
        image=image,
        command=['sh'],
        arguments=['-c', ' && '.join(commands)],
        container_kwargs={'image_pull_policy': 'IfNotPresent'},
        pvolumes={'/workspace': volume_op.volume}
    )

    return op


In [6]:
commands = [
        "ls",
        "pwd",
        'which vq-vae_1d',
        "pwd",
        "cd /home/jovyan/vq-vae_1d",
        "pwd",
        "ls"
    ]
train=dsl.ContainerOp(
        name='VQ-VAE',
        image='zaku.sys.es.osaka-u.ac.jp:10081/ohmori/vq-vae_1d:v0.0.0',
        command=['sh'],
        arguments=['-c', ' && '.join(commands)],
        # command=['python3', '~/workspace/vq-vae_1d/script/train.py'],
        # arguments=[
        #     '--data_type', data_type,
        #     '--f_name', f_name,
        #     '--num_hiddens', num_hiddens,
        #     '--num_residual_hiddens', num_residual_hiddens,
        #     '--num_residual_layers', num_residual_layers,
        #     '--embedding_dim', embedding_dim,
        #     '--num_embeddings', num_embeddings,
        #     '--commitment_cost', commitment_cost,
        #     '--decay', decay,
        #     '--epoch', epoch,
        #     '--lr', lr,
        #     '--batch_size', batch_size
        # ],
#         pvolumes={'/workspace': git_clone.pvolume},
        file_outputs={
            'MLPipeline UI metadata': '/mlpipeline-ui-metadata.json'
        }
    )

In [12]:
train.add_volume?

Signature: train.add_volume(volume)
Docstring:
Add K8s volume to the container

Args:
  volume: Kubernetes volumes
  For detailed spec, check volume definition
  https://github.com/kubernetes-client/python/blob/master/kubernetes/client/models/v1_volume.py
File:      /opt/conda/lib/python3.7/site-packages/kfp/dsl/_container_op.py
Type:      method


In [13]:
k8s_client.V1Volume?

Init signature:
k8s_client.V1Volume(
    aws_elastic_block_store=None,
    azure_disk=None,
    azure_file=None,
    cephfs=None,
    cinder=None,
    config_map=None,
    csi=None,
    downward_api=None,
    empty_dir=None,
    fc=None,
    flex_volume=None,
    flocker=None,
    gce_persistent_disk=None,
    git_repo=None,
    glusterfs=None,
    host_path=None,
    iscsi=None,
    name=None,
    nfs=None,
    persistent_volume_claim=None,
    photon_persistent_disk=None,
    portworx_volume=None,
    projected=None,
    quobyte=None,
    rbd=None,
    scale_io=None,
    secret=None,
    storageos=None,
    vsphere_volume=None,
)
Docstring:     
NOTE: This class is auto generated by OpenAPI Generator.
Ref: https://openapi-generator.tech

Do not edit the class manually.
Init docstring: V1Volume - a model defined in OpenAPI
File:           /opt/conda/lib/python3.7/site-packages/kubernetes/client/models/v1_volume.py
Type:           type
Subclasses:     PipelineVolume


In [14]:
dsl.PipelineVolume?

Init signature:
dsl.PipelineVolume(
    pvc: str = None,
    volume: kubernetes.client.models.v1_volume.V1Volume = None,
    **kwargs,
)
Docstring:     
Representing a volume that is passed between pipeline operators and is
to be mounted by a ContainerOp or its inherited type.

A PipelineVolume object can be used as an extention of the pipeline
function's filesystem. It may then be passed between ContainerOps,
exposing dependencies.
Init docstring:
Create a new instance of PipelineVolume.

Args:
    pvc: The name of an existing PVC
    volume: Create a deep copy out of a V1Volume or PipelineVolume
        with no deps
Raises:
    ValueError: if volume is not None and kwargs is not None
                if pvc is not None and kwargs.pop("name") is not None
File:           /opt/conda/lib/python3.7/site-packages/kfp/dsl/_pipeline_volume.py
Type:           type
Subclasses:     


In [15]:
dsl.VolumeOp?

Init signature:
dsl.VolumeOp(
    resource_name: str = None,
    size: str = None,
    storage_class: str = None,
    modes: List[str] = None,
    annotations: Dict[str, str] = None,
    data_source=None,
    volume_name=None,
    **kwargs,
)
Docstring:     
Represents an op which will be translated into a resource template
which will be creating a PVC.
Init docstring:
Create a new instance of VolumeOp.

Args:
    resource_name: A desired name for the PVC which will be created
    size: The size of the PVC which will be created
    storage_class: The storage class to use for the dynamically created
        PVC
    modes: The access modes for the PVC
    annotations: Annotations to be patched in the PVC
    data_source: May be a V1TypedLocalObjectReference, and then it is
        used in the data_source field of the PVC as is. Can also be a
        string/PipelineParam, and in that case it will be used as a
        VolumeSnapshot name (Alpha feature)
    volume_name: VolumeName is the b